In [20]:
import os
import sys
sys.path.append('../src')

from openai import OpenAI
from dotenv import load_dotenv
from agents import BaseAgent, Agent, MultiAgent
from pydantic import BaseModel

In [21]:
load_dotenv()

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_BASE_URL"),
)


In [33]:
# Top level agent
general_agent = Agent(base_agent = BaseAgent(client),
           name = "general agent",
           instructions= "Determine which agent is best suited to handle the user's request, and transfer the conversation to that agent and output the transfer message.",
           handoff_description = "Call this agent if you don't know how to answer the user's question or do not has access to the necessary information.")

In [34]:
class AgentDesc(BaseModel):
    name: str
    instructions: str
    handoff_description: str
    
def create_new_agent(input: str)->Agent:
    """ 
    If there is no other agent or tool can handle the input, call this function to create a new agent.
    """

    model = "gpt-4o-mini"

    completion = client.beta.chat.completions.parse(
    model=model,
    messages=[
        {"role": "system", "content": """ 
         Create a new agent. The agent should be able to handle user request. return the agent description.

        ## name example
        
         science agent (only english characters and spaces are allowed)
         
        ## instructions example
         
        You are a Science Agent As a science agent, you have a strong understanding of physics, chemistry, and biology. You can answer a variety of questions in these fields. Your aim is to make science interesting and easy to understand for everyone. If a question is outside your expertise, you can refer the user to another agent.
         
        ## handoff_description example
        
        Call this agent if you have a question related to science, physics, chemistry, or biology. This agent can provide detailed explanations, clarify complex concepts, and offer insights into the natural world.
         
         """},
        {"role": "user", "content": input},
    ],
    response_format=AgentDesc,
)

    event = completion.choices[0].message.parsed

    new_agent = Agent(base_agent = BaseAgent(client),name=event.name, instructions=event.instructions, handoff_description=event.handoff_description)

    general_agent.add_handoff(new_agent)
    new_agent.add_handoff(general_agent)

    return "#CreateNewAgent#"
    


In [35]:
general_agent.add_tools([create_new_agent])

In [36]:
multi_agent = MultiAgent(start_agent=general_agent)

In [37]:
multi_agent.chat([{ "role": "user", "content": "how to ride a bike?"}],show_details=True)

[{'role': 'assistant',
  'tool_calls': [{'id': 'call_vw1KqBnN9ccOS6FYT5RTVNeI',
    'function': {'arguments': '{"input":"I\'m looking for tips and instructions on how to ride a bike."}',
     'name': 'create_new_agent'},
    'type': 'function'}],
  'type': 'tool',
  'sender': 'general_agent'},
 {'role': 'tool',
  'content': '#CreateNewAgent#',
  'tool_call_id': 'call_vw1KqBnN9ccOS6FYT5RTVNeI',
  'type': 'tool',
  'sender': 'general_agent'},
 {'role': 'assistant',
  'content': 'I’ll transfer you to an agent that specializes in providing guidance on riding a bike. Please hold on for a moment.',
  'sender': 'general_agent'},
 {'role': 'assistant',
  'tool_calls': [{'id': 'call_OnT8mIbuVYTWRLU2fAMcyJMo',
    'function': {'arguments': '{"message": "how to ride a bike?"}',
     'name': 'Bike_Riding_Agent'},
    'type': 'function'}],
  'type': 'handoff',
  'sender': 'general_agent'},
 {'role': 'tool',
  'handoff': 'general_agent -> Bike_Riding_Agent',
  'content': 'Handing off to Bike_Riding_

In [38]:
multi_agent.chat([{ "role": "user", "content": "what kind of bike should I buy if i want to ride in the mountain?"}],show_details=True)

[{'role': 'assistant',
  'content': "If you're looking to ride in the mountains, you should consider purchasing a mountain bike (MTB). Mountain bikes are designed specifically for off-road conditions and have features that make them suitable for rough terrains. Here are some key factors to consider when choosing a mountain bike:\n\n1. **Frame Material**:\n   - **Aluminum**: Lightweight and affordable.\n   - **Carbon Fiber**: Lighter and offers better performance but more expensive.\n   - **Steel**: Durable and comfortable, but heavier.\n\n2. **Suspension Type**:\n   - **Hardtail**: Has front suspension only, which is great for climbing and smoother trails.\n   - **Full Suspension**: Has both front and rear suspension, providing more comfort on rough and technical trails but can be heavier and more expensive.\n\n3. **Wheel Size**:\n   - **26 inches**: Offers more agility but less stability on rough terrains.\n   - **27.5 inches**: A good compromise between agility and stability.\n   - *

In [39]:
multi_agent.chat([{ "role": "user", "content": "can you explain the theory of relativity?"}],show_details=True)

[{'role': 'assistant',
  'tool_calls': [{'id': 'call_41ADdWOZPcuhehUGG1ACxkNv',
    'function': {'arguments': '{"message": "can you explain the theory of relativity?"}',
     'name': 'general_agent'},
    'type': 'function'}],
  'type': 'handoff',
  'sender': 'Bike_Riding_Agent'},
 {'role': 'tool',
  'handoff': 'Bike_Riding_Agent -> general_agent',
  'content': 'Handing off to general_agent',
  'tool_call_id': 'call_41ADdWOZPcuhehUGG1ACxkNv',
  'message': 'can you explain the theory of relativity?',
  'type': 'handoff',
  'sender': 'general_agent'},
 {'role': 'assistant',
  'tool_calls': [{'id': 'call_hqGPYHNtFLYu49oOQarqVt3J',
    'function': {'arguments': '{"input":"Can you explain the theory of relativity?"}',
     'name': 'create_new_agent'},
    'type': 'function'}],
  'type': 'tool',
  'sender': 'general_agent'},
 {'role': 'tool',
  'content': '#CreateNewAgent#',
  'tool_call_id': 'call_hqGPYHNtFLYu49oOQarqVt3J',
  'type': 'tool',
  'sender': 'general_agent'},
 {'role': 'assistan

In [40]:
general_agent.handoff_agent_schemas

[{'type': 'function',
  'function': {'name': 'Bike_Riding_Agent',
   'description': 'You are a Bike Riding Agent. As a bike riding agent, you have extensive knowledge about the different types of bicycles, riding techniques, safety tips, and maintenance practices. Your mission is to help users learn how to ride a bike confidently, explaining the basics as well as advanced techniques for different terrains. If a question is outside your expertise, you can refer the user to another agent.',
   'parameters': {'type': 'object',
    'properties': {'message': {'type': 'string',
      'description': 'The message to send to the agent.'}},
    'required': ['message']}}},
 {'type': 'function',
  'function': {'name': 'Science_Agent',
   'description': 'You are a Science Agent. As a science agent, you have a strong understanding of physics, chemistry, and biology. You can answer a variety of questions in these fields. Your aim is to make science interesting and easy to understand for everyone. If 

In [41]:
multi_agent.chat([{ "role": "user", "content": "what is the speed of light?"}],show_details=True)

[{'role': 'assistant',
  'content': 'The speed of light in a vacuum is approximately \\(299,792,458\\) meters per second, often rounded to \\(3.00 \\times 10^8\\) meters per second. This constant is denoted by the letter "c" and is fundamental in physics, particularly in the theory of relativity developed by Albert Einstein. Light travels at this speed in a vacuum, but it can slow down when passing through different mediums, such as water or glass.',
  'sender': 'Science_Agent'}]

In [42]:
multi_agent.chat([{ "role": "user", "content": "what to eat for breakfast?"}],show_details=True)

[{'role': 'assistant',
  'tool_calls': [{'id': 'call_C48jvHHRBlDYoK9KqOPmj39Z',
    'function': {'arguments': '{"message": "what to eat for breakfast?"}',
     'name': 'general_agent'},
    'type': 'function'}],
  'type': 'handoff',
  'sender': 'Science_Agent'},
 {'role': 'tool',
  'handoff': 'Science_Agent -> general_agent',
  'content': 'Handing off to general_agent',
  'tool_call_id': 'call_C48jvHHRBlDYoK9KqOPmj39Z',
  'message': 'what to eat for breakfast?',
  'type': 'handoff',
  'sender': 'general_agent'},
 {'role': 'assistant',
  'tool_calls': [{'id': 'call_3ArafLz4LEddkBXmbjzcpnwk',
    'function': {'arguments': '{"input":"What to eat for breakfast?"}',
     'name': 'create_new_agent'},
    'type': 'function'}],
  'type': 'tool',
  'sender': 'general_agent'},
 {'role': 'tool',
  'content': '#CreateNewAgent#',
  'tool_call_id': 'call_3ArafLz4LEddkBXmbjzcpnwk',
  'type': 'tool',
  'sender': 'general_agent'},
 {'role': 'assistant',
  'content': "I'm transferring your request to ou